# Paramètres de la pipeline

In [1]:
code_departement = "093"
logs_level = "WARNING"

# Imports & setup

In [2]:
import geopandas as gpd

from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools
from potentiel_solaire.constants import CRS, ALGORITHME_FOLDER
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, \
    get_topo_buildings_of_interest
from potentiel_solaire.sources.schools_establishments import extract_schools_establishments, \
    get_schools_establishments_of_interest
from potentiel_solaire.features.solar_potential import calculate_solar_potential
from potentiel_solaire.aggregate import aggregate_solar_potential_by
from potentiel_solaire.logger import get_logger

logger = get_logger()
logger.setLevel(logs_level)

# Extraction des données sources

### Etablissements scolaires

In [3]:
schools_establishments_path = extract_schools_establishments()
print(f"Annuaire des établissements scolaires extrait ici: {schools_establishments_path}")

Annuaire des établissements scolaires extrait ici: C:\Developpement\13_potentiel_solaire\algorithme\data\fr-en-annuaire-education.geojson


### BD TOPO

In [4]:
bd_topo_path = extract_bd_topo(code_departement=code_departement)
print(f"BD TOPO extraite ici: {bd_topo_path}")

BD TOPO extraite ici: C:\Developpement\13_potentiel_solaire\algorithme\data\BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15\BDTOPO\1_DONNEES_LIVRAISON_2024-12-00134\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg


### BD PCI

In [5]:
#TODO: recuperer les données PCI pour le département

# Filtre des données sur le périmètre du calcul

### Etablissements scolaires


In [6]:
schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=schools_establishments_path,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=CRS
)
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

Nb d'établissements scolaires: 1130


### Zone d'intérêt géographique

In [7]:
codes_commune = schools_establishments["code_commune"].unique()
communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(CRS)
communes = communes[communes.code_insee.isin(codes_commune)]
geom_of_interest = communes.dissolve()[["geometry"]]

### Zones d'éducations

In [8]:
educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=CRS
)
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\io\file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_numerotee_ou_nommee', 'itineraire_au

Nb de zones d'éducations:  1088


### Bâtiments

In [9]:
# TODO : add missing buildings with BD PCI
buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=CRS
)
nb_buildings = buildings.shape[0]
print("Nb de batiments: ", nb_buildings)

C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\io\file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_numerotee_ou_nommee', 'itineraire_au

Nb de batiments:  351576


# Détermination des bâtiments scolaires

In [10]:
schools_buildings = attach_buildings_to_schools(
        schools_establishments=schools_establishments,
        educational_zones=educational_zones,
        buildings=buildings
)

C:\Developpement\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


# Calcul des attributs utiles pour le potentiel solaire

In [11]:
# TODO: Fausses valeurs pour le moment
solar_potential_of_schools_buildings = calculate_solar_potential(
    schools_buildings=schools_buildings,
)

# Checks sur la qualité des données & calculs

In [12]:
nb_schools_with_buildings = len(schools_buildings.identifiant_de_l_etablissement.unique())
print("Nb d'établissements scolaires avec des batiments: {} ({}%)".format(
    nb_schools_with_buildings,
    round(100 * nb_schools_with_buildings / nb_schools)
))

Nb d'établissements scolaires avec des batiments: 445 (39%)


# Aggrégations

### Par établissement scolaire

In [13]:
results_by_school = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "identifiant_de_l_etablissement",
        "nom_etablissement",
        "type_etablissement",
        "libelle_nature",
        "code_commune",
        "nom_commune",
        "code_departement",
        "libelle_departement",
        "code_region",
        "libelle_region",
    ]
)

### Par commune

In [14]:
results_by_commune = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "code_commune",
        "nom_commune",
        "code_departement",
        "libelle_departement",
        "code_region",
        "libelle_region",
    ]
)
# TODO : la geometrie de la commune est fausse

### Par département

In [15]:
results_by_departement = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "code_departement",
        "libelle_departement",
        "code_region",
        "libelle_region",
    ]
)
# TODO : la geometrie du departement est fausse

### Par région

In [16]:
# TODO : demande de se connecter au bucket avec les resultats existants pour aggreger 
# TODO : ou de faire le calcul pour tous les departements d une region

# Sauvegarde des fichiers

In [17]:
output_folder = ALGORITHME_FOLDER.parent / "results" / f"D{code_departement}"
output_folder.mkdir(exist_ok=True, parents=True)

# TODO : a voir cote front sil faut changer le format de mise a disposition
results_by_school.to_file(output_folder / f"D{code_departement}_ecoles.geojson", driver="GeoJSON")
results_by_commune.to_file(output_folder / f"D{code_departement}_communes.geojson", driver="GeoJSON")
results_by_departement.to_file(output_folder / f"D{code_departement}_departement.geojson", driver="GeoJSON")